In [ ]:
!pip install openai-whisper
!pip install torchaudio


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 14.4 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.0/865.0 MB 18.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 29.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 37.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 40.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 17.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 39.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━

In [2]:
import torch
import whisper
import torchaudio
import librosa
import soundfile
import numpy as np
import os
import json
import random
import joblib
from transformers import pipeline

ModuleNotFoundError: No module named 'torchaudio'

In [ ]:
# Load Whisper model
whisper_model = whisper.load_model("small")

# Load Hindi to English translator
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-hi-en")

# Load trained MLP emotion classifiers
hindi_emotion_model = joblib.load("mlp_classifier_hindi.model")
english_emotion_model = joblib.load("mlp_classifier_english.model")

100%|████████████████████████████████████████| 461M/461M [00:03<00:00, 155MiB/s]
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/304M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/304M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

...

source.spm:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


FileNotFoundError: [Errno 2] No such file or directory: 'mlp_classifier_hindi.model'

In [ ]:
def transcribe(wav_file):
    """Transcribes audio using Whisper and detects language."""
    transcription = whisper_model.transcribe(wav_file)["text"]
    detected_language = "Hindi" if any('\u0900' <= char <= '\u097F' for char in transcription) else "English"

    if detected_language == "Hindi":
        english_translation = translator(transcription)[0]['translation_text']
        return detected_language, transcription, english_translation
    return detected_language, transcription, None

In [ ]:
def extract_features(file_name):
    """
    Extracts MFCC, Chroma, and Mel Spectrogram features (matches training process).
    """
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate

        # Compute Short-Term Fourier Transform (STFT) if required
        stft = np.abs(librosa.stft(X))

        # Initialize feature vector
        result = np.array([])

        # Extract MFCC
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))

        # Extract Chroma features
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
        result = np.hstack((result, chroma))

        # Extract Mel Spectrogram features
        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
        result = np.hstack((result, mel))

    return np.expand_dims(result, axis=0)  # Match input shape for model


In [ ]:
def detect_emotion(audio_file, language):
    """
    Predicts emotion using the trained MLP model based on the detected language.
    """
    features = extract_features(audio_file)
    model = hindi_emotion_model if language == "Hindi" else english_emotion_model
    emotion_label = model.predict(features)[0]  # Extract single prediction

    return emotion_label


In [ ]:
def process_audio(audio_file):
    """Runs transcription, emotion detection, and emotion management pipeline."""
    print("\nProcessing audio file...")

    # Step 1: Transcription and Translation
    language, transcription, translation = transcribe(audio_file)
    if language is None:
        print("Error: Could not process audio.")
        return None

    # Step 2: Emotion Detection
    emotion = detect_emotion(audio_file, language)

    # Step 3: Emotion Management (Now returns the intervention)
    management_response = emotion_management(emotion)

    # Print Results
    print("\n-----RESULTS-----")
    print(f"Detected Language: {language}")
    print(f"Transcription: {transcription}")
    if translation:
        print(f"Translation: {translation}")
    print(f"Emotion: {emotion}")

    return language, transcription, translation, emotion


In [ ]:
if __name__ == "__main__":
  load_q_table()
  audio_path = "/content/1.1.sad-06.wav"  # Replace with the actual path of your test file
  process_audio(audio_path)